A simple notebook to do MIRI coordinate transforms.
This requires that the JWST pipeline be installed.

In [1]:
import numpy as np
import pdb as pdb
from astropy.modeling import models
from asdf import AsdfFile
from jwst import datamodels
from jwst.assign_wcs import miri

### Imager transforms: import the mirim tools code ###

In [2]:
import imager.mirim_tools as mirim_tools

Define the transform from imager x,y pixel coordinate
to V2,V3 coordinates for a given filter.  Pixel coordinates are 0-indexed,
where (0,0) is the center of the bottom left detector pixel.  (I.e., (4,0) 
is the center of the bottom left science pixel).
Use F770W which defines the boresight.

In [3]:
xform=mirim_tools.xytov2v3('F770W')

Do a forward transform, in this case for the reference location (where the
telescope would point in absence of a dither offset).  It should give V2,V3=-453.363,-374.069 arcsec

In [4]:
x,y=692.5,511.5

In [5]:
v2,v3=xform(x,y)

In [6]:
print(v2,v3)

-453.3633633490669 -374.0686305853809


Do a reverse transform

In [7]:
x_back,y_back=xform.inverse(v2,v3)

In [8]:
print(x_back,y_back)

692.496064329321 511.4999501761038


### Now do a conversion to Ideal coordinates using the SIAF apertures ###

Let's work out where v2,v3=-414.33, -400.69 is for the LRS slit

In [9]:
v2,v3=-414.33, -400.69

In [10]:
xideal,yideal=mirim_tools.v2v3toIdeal(v2,v3,'MIRIM_SLIT')

In [11]:
print(xideal,yideal)

0.0 0.0


It's 0,0, which makes sense since this was the MIRIM_SLIT reference point.

Now try the lower-left corner of the LRS slit:

In [12]:
v2,v3=-411.99, -401.14

In [13]:
xideal,yideal=mirim_tools.v2v3toIdeal(v2,v3,'MIRIM_SLIT')

In [14]:
print(xideal,yideal)

-2.367430600416094 -0.27087331395574166


APT requires Ideal coordinate offsets from the reference point

If we wanted to see where an offset of XIdeal,YIdeal=10,0 would land a star on the imager detector:

In [15]:
xideal,yideal=10,0

In [16]:
v2,v3=mirim_tools.Idealtov2v3(xideal,yideal,'MIRIM_ILLUM')

In [17]:
print(v2,v3)

-463.333221220753 -373.29278941730036


In [18]:
xform=mirim_tools.xytov2v3('F770W')

In [19]:
x,y=xform.inverse(v2,v3)

In [20]:
print(x,y)

782.343756766181 511.49624592242793


### Now we'll do an MRS transform.  Import the mirimrs tools code ###

In [21]:
import mrs.mmrs_tools as mmrs_tools

Define the transform from MRS detector x,y pixel coordinates to local alpha,beta,lambda coordinates.
Pixel coordinates are 0-indexed, where (0,0) is the center of the bottom left detector pixel.
We'll do the transform for Ch 1A.

In [22]:
xytoabl=mmrs_tools.xytoabl('1A')

Do a forward transform, in this case for the MRS reference location (where the telescope points in absence of a dither offset).  It should give V2,V3=-503.654,-318.742

In [23]:
x,y=27.31,511.0

In [24]:
a,b,l=xytoabl(x,y)

In [25]:
print(a,b,l)

-8.209223011187983e-05 0.0 5.344367253905783


Now do the transform from local alpha,beta coordinates to V2,V3

In [26]:
abtov2v3=mmrs_tools.abtov2v3('1A')

In [27]:
v2,v3=abtov2v3(a,b)

In [28]:
print(v2,v3)

-503.65455315000435 -318.74243867549205


Now do reverse MRS transforms

In [29]:
a_back,b_back=abtov2v3.inverse(v2,v3)

In [30]:
print(a_back,b_back)

0.00015184635690346227 0.0003153709700143281


In [31]:
x_back,y_back=xytoabl.inverse(a_back,b_back,l)

In [32]:
print(x_back,y_back)

27.31146890194275 510.9989221257771
